In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import statsmodels.api as sm
import itertools
import pandas as pd
import collections
from tqdm import tqdm
from joblib import Parallel, delayed
from operator import itemgetter
from itertools import chain

from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

In [2]:
# https://www.ue.katowice.pl/fileadmin/_migrated/content_uploads/7_B.Fraczek_Analiza_Czynnikow....pdf
df = pd.read_excel("datasets/cleaned.xlsx", sheet_name = ["Polska"])["Polska"]
df = df.fillna(df.mean())

In [3]:
df.set_index("Rok", inplace=True)

In [67]:
# Kolumny z odpowiednią wartością wariancji 
cols = df.columns[df.std() / df.mean() > 0.2].values
cols: list = cols.tolist()
y_col = "Procent dochodu rozporządzalnego przeznaczany na oszczędności" 
cols.remove("Procent dochodu rozporządzalnego przeznaczany na oszczędności")
X_cols = cols

In [68]:
R = df.corr().loc[X_cols, X_cols] # Macierz korelacji zmiennych
R_0 = df.corr().loc[X_cols, y_col] # Macierz koralacji zmiennyh i zmiennej objaśniającej

In [94]:
H = []
for k, combination in enumerate(list(chain.from_iterable([list(itertools.combinations(X_cols, n)) for n in range(1, len(X_cols) + 1)]))):
    h = []
    for var in combination:
        r = R_0[var]
        denominator = np.sum(np.abs(R.loc[list(combination), var]))
        h.append(r**2/denominator)
    H.append((combination, sum(h)))

In [95]:
best = max(H, key=itemgetter(1))

In [96]:
len(H), len(best[0])

(1023, 2)

In [97]:
best

(('Średnie stopy procentowe w danym roku',
  'Cena: olej jadalny rafinowany rzepakowy konfekcjonowany - za 1l'),
 0.8643742743209077)